In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # es para enseñar todas las columnas del df
pd.set_option('display.max_rows', None)     # es para enseñar todas las filas del df


import warnings
warnings.simplefilter('ignore')   # es para quitar warnings

import mysql.connector as conn

## Importo todos los archivos

In [2]:
# Importo todos los csv

catergory =pd.read_csv('../data/new_data/category.csv')

language =pd.read_csv('../data/new_data/language.csv')

actor =pd.read_csv('../data/new_data/actor.csv')

actor_film =pd.read_csv('../data/new_data/actor_film.csv')

film =pd.read_csv('../data/new_data/film.csv')

inventory =pd.read_csv('../data/new_data/inventory.csv')

rental =pd.read_csv('../data/new_data/rental.csv')

store =pd.read_csv('../data/new_data/store.csv')

customer =pd.read_csv('../data/new_data/customer.csv')

staff =pd.read_csv('../data/new_data/staff.csv')



## Crear base de datos en mySQL

### Creo conexion y base de datos

In [3]:
# Conecta a una base de datos mysql y activa el cursor/motor

password = "".join(open('../text/mysql_pass.txt', "r").readlines()) # Me traigo el password de mysql

crea_db=conn.connect(host='localhost', user='root', passwd=password)

cursor=crea_db.cursor()



In [4]:
# Crear base de datos

cursor.execute('drop database if exists blockbuster;')
cursor.execute('create database blockbuster;')

In [5]:
db=conn.connect(host='localhost', user='root', passwd=password, database='blockbuster')

cursor=db.cursor()

In [6]:
# check si me creo la base de datos

cursor.execute('show databases;')

for x in cursor:
    print(x)

('blockbuster',)
('information_schema',)
('mydb',)
('mysql',)
('performance_schema',)
('publications',)
('sys',)


## Creo tablas con sus caracteristicas

In [7]:
# Crear tabla 'actor'


cursor.execute('drop table if exists actor;')



tabla='''

        create table actor (
            
            actor_id int primary key,
            first_name varchar(50),
            last_name  varchar(50)
        );

'''


cursor.execute(tabla)

In [8]:
# Crear tabla 'language'

cursor.execute('drop table if exists language')

tabla= '''
            create table language (
            
            language_id int primary key,
            name varchar(50)         
            
            );
'''
cursor.execute(tabla)

In [9]:
# Crear tabla 'category'

cursor.execute('drop table if exists category')

tabla = '''
            create table category(
            category_id int primary key,
            name varchar(50)
            
            );
'''

cursor.execute(tabla)

In [10]:
# Crear tabla 'film'

cursor.execute('drop table if exists film')

tabla = '''
            create table film(
            
            film_id int primary key,
            title varchar(50),
            description varchar(250),
            release_year int,
            rental_duration int,
            rental_rate float,
            length int,
            replacement_cost float,
            special_features varchar(250),
            rating varchar(50),
            language_id int,
            category_id int
            
            );
'''

cursor.execute(tabla)

In [11]:
# Crear tabla 'inventory'

cursor.execute('drop table if exists inventory')

tabla = '''
            create table inventory(
            
            inventory_id int primary key,
            film_id int,
            store_id int
            
            );
'''

cursor.execute(tabla)

In [12]:
# Crear tabla 'rental'

cursor.execute('drop table if exists rental')

tabla = '''
            create table rental(
            
            rental_id int primary key,
            rental_date datetime,
            return_date datetime,
            inventory_id int,
            customer_id int,
            staff_id int
            
            );
'''

cursor.execute(tabla)

In [13]:
# Creo tabla 'store'

cursor.execute('drop table if exists store')

tabla = ''' 
            create table store(
            
            store_id int primary key,
            name varchar(50),
            city varchar(50)
    
            );

'''

cursor.execute(tabla)

In [14]:
# Creo tabla 'customer'

cursor.execute('drop table if exists customer')

tabla = '''
            create table customer(
            
            customer_id int primary key,
            firstname varchar(50),
            lastname varchar(50),
            email varchar(100)
            
            );


'''
cursor.execute(tabla)

In [15]:
# Creo tabla 'staff'

cursor.execute('drop table if exists staff')

tabla = '''
            create table staff(
            
            staff_id int primary key,
            firstname varchar(50),
            lastname varchar(50),
            salary int
            
            );

'''

cursor.execute(tabla)

In [16]:
# Creo la tabla 'actor_film' que hará de tabla intermedia en la relación many to many entre 'actor' y 'film'

cursor.execute('drop table if exists actor_film')

tabla = ''' 
            create table actor_film(
            
            actor_id int not null,
            film_id int not null,
            primary key(actor_id, film_id)
            
            
            );


'''

cursor.execute(tabla)

## Introduzco datos en las tablas

In [17]:
#  insert into tabla (columnas) values (valores);

def insert(df, table):

    for i in range(len(df)):
        
        insert_query=f"insert into {table} ({','.join(df.columns)}) values {tuple(df.iloc[i].values)};"
    
        cursor.execute(insert_query)
    
    
    db.commit()
    print('Comiteado!')

In [18]:
# Cargo datos a 'category'

insert(catergory, 'category')

Comiteado!


In [19]:
# Cargo datos a 'language'

insert(language, 'language')

Comiteado!


In [20]:
# Cargo datos a 'actor'

insert(actor, 'actor')

Comiteado!


In [21]:
# Cargo datos a 'film'

insert(film, 'film')

Comiteado!


In [22]:
# Cargo datos a 'inventory'

insert(inventory, 'inventory')

Comiteado!


In [23]:
# Cargo datos a 'rental'

insert(rental, 'rental')

Comiteado!


In [24]:
# Cargo datos a 'store'

insert(store, 'store')

Comiteado!


In [25]:
# Cargo datos a 'customer'

insert(customer, 'customer')

Comiteado!


In [26]:
# Cargo datos a 'staff'

insert(staff, 'staff')

Comiteado!


In [27]:
# Cargo datos a 'actor_film'

insert(actor_film, 'actor_film')

Comiteado!


## Creo realaciones entre tablas

In [28]:
# Creo todas las relaciones de una vez

# Me genera un error
# DatabaseError: 2014 (HY000): Commands out of sync; you can't run this command now
# No se que mierda del buffer con las queries

relation = '''
        
-- relaciones entre tabla 'film' y 'category' y 'language'
        
        
            alter table film
            add foreign key(category_id)
            references category(category_id)
            on delete set null;
        
            alter table film
            add foreign key(language_id)
            references language(language_id)
            on delete set null;
            
            
-- relaciones entre tabla 'inventory' y 'film' y 'store'

            alter table inventory
            add foreign key(film_id)
            references film(film_id)
            on delete set null;



            alter table inventory
            add foreign key(store_id)
            references store(store_id)
            on delete set null;
            
-- relaciones entre tabla 'rental' y 'inventory', 'customer' y 'staff'

            alter table rental
            add foreign key(inventory_id)
            references inventory(inventory_id)
            on delete set null;



            alter table rental
            add foreign key(customer_id)
            references customer(customer_id)
            on delete set null;
            
            alter table rental
            add foreign key(staff_id)
            references staff(staff_id)
            on delete set null;


-- relaciones entre tabla 'actor_film' y 'actor' y 'film'
-- relacion many to many


                alter table actor_film
                add constraint 
                fk_actorfilm__actor foreign key(actor_id) references actor(actor_id);

                alter table actor_film
                add constraint 
                fk_actorfilm__film foreign key(film_id) references film(film_id);

        
'''

# cursor.execute(relation)